## Chapter 1.5: Bringing all together: Real time plotting- Real time DataFrames


Now that we have seen how Qgrid is integrated with GemPy we can the interactivity of vtk and the df. Lets start over a `geo_data` object.


In [3]:
geo_model.grid.extent

array([    0.,  1000.,     0.,  1000., -1000.,     0.])

### Using Qgrid for interactive DataFrames

From its documentation :

> Qgrid is a Jupyter notebook widget which uses SlickGrid to render pandas DataFrames within a Jupyter notebook. This allows you to explore your DataFrames with intuitive scrolling, sorting, and filtering controls, as well as edit your DataFrames by double clicking cells.

In practice these library allows us to use `pandas.DataFrames` in Jupyter Notebooks as an excel table. Be aware that `Qgrid` requires of enabling nbextensions so it is important to test that the installation (https://github.com/quantopian/qgrid) was successful before trying to execute the rest of the notebook.

By calling `gempy.interactive_df_open` and passing the dataframe we want to modify we can manipulate the *df* interactivelly. But first we need to define some *formations* since basement
does not get involved in the computation of the model:

In [4]:
gp.set_formation_names(geo_model, ['layer1', 'layer2'])

,formation,series,id,isBasement
0,layer1,Default series,1,False
1,layer2,Default series,2,True


In [5]:
geo_model.series.add_series(['foo'])
geo_model.update_from_series()
geo_model.series.df['BottomRelation']

Default series    Erosion
foo               Erosion
Name: BottomRelation, dtype: category
Categories (2, object): [Erosion, Onlap]

In [6]:
gp.set_interactive_df(geo_model)

In [7]:
geo_model.qi.set_interactive_df('formations')

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [8]:
geo_model.formations

,formation,series,id,isBasement
0,layer1,Default series,1,False
1,layer2,Default series,2,True


## Qgrid Series

In [9]:
geo_model.series

,order_series,BottomRelation
Default series,1,Erosion
foo,2,Erosion


In [10]:
geo_model.qi.set_interactive_df('series')

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [11]:
geo_model.update_from_series()

## Faults

In [12]:
sgs

NameError: name 'sgs' is not defined

In [ ]:
pn.DataFrame(index=pn.CategoricalIndex(['Default series']), columns=['isFault', 'isFinite'], dtype=bool)

In [ ]:
geo_model.faults

In [ ]:
qgrid_widget = qgrid.show_grid(geo_model.faults.df,
        show_toolbar=True,
        column_options={'editable': True},
        column_definitions={})

In [ ]:
qgrid_widget

Couple of things to notice:

1) The *df* does not display all the changes that happen when we modify a field (i.e. if we change the formation to layer 1, the formation_number should be 1). However, under the hood these changes are happening and if you save the *df* you will see all as it is supossed to be.

2) The *df* in geo_data does not get updated in real time. If you want to save the changes you need to use the following function. This returns a copy of the modified *df*

In [ ]:
# To set the new df we use the function set_surface_points
gp.set_surface_points(geo_data, gp.interactive_df_change_df(geo_data))

In [ ]:
%matplotlib notebook
gp.plotting.plot_data(geo_data, direction='x')

Cool isn't it. Now writing new data and in special editing it is not so tiresome... however... we are not in 2017. Welcome to interactive *df* with interactive plotting:

### Bringing all together: Real time plotting- Real time DataFrames

Now that we have seen how Qgrid is integrated with GemPy we can the interactivity of vtk and the df. Lets start over a `geo_data` object.


In [ ]:
# These two lines are necessary only if gempy is not installed
import sys, os
sys.path.append("../..")
import gempy as gp
geo_data = gp.InputData([0, 1000, 0, 1000, -1000, 0], [50, 50, 50])

Next we can directly compile an `interp_data` objects. However, theano requires at least an interface and orientation. If we try to compile an empty `geo_data`, `GemPy` will create default points at the center of the extent. Let's try it out:

In [ ]:
interp_data = gp.InterpolatorData(geo_data, compile_theano=True,
                                  theano_optimizer='fast_compile')

If we try to compute the model right now we will raise an error since we do not have enough data. What we can do is start the real time visualization of vtk. Since the last update the vtk visualization quite a few new options and hence it just became its own class to facilitate its manipulation.

In [ ]:
# We instanciate a vtkPlot objecth
vtk_plot = gp.plotting.vtkPlot(geo_data)

# We call the real time plotter passing the interp_data
vtk_plot.plot_surfaces_3D_real_time(interp_data, size=(1920, 2160))

In the plot you can see both default points and you can interact with them by drag and drop as usual. A very important new feature is the possibility to press **H** or **P** to go back to the IPhython kernel. To come back to the vtk interaction use the following method:

In [ ]:
vtk_plot.resume()

At this point that you have seen how to come back to our Python script is when Qgrid take over. Using the following method we will open an interactive *df* as seen above that will interact with the vtk plot in real time:

In [ ]:
vtk_plot.observe_df(geo_data, itype='surface_points')

At this point you can create any type of model, going back and forth between the vtk real time interaction and the *df*. As an example and preparation of the next tutorial I create a graben structure

#### Example Interaction: Graben


In [ ]:
# These two lines are necessary only if gempy is not installed
import sys, os
sys.path.append("../..")

# Importing gempy
import gempy as gp

# Embedding matplotlib figures into the notebooks
#%matplotlib inline


# Aux imports
import numpy as np
import pandas as pn

In [ ]:
try:
    geo_data = gp.read_pickle('backup1.pickle')
except FileNotFoundError:
    print('Pickle Not found')
    geo_data = gp.InputData([0, 1000, 0, 1000, -1000, 0], [50, 50, 50])

In [ ]:
gp.set_series(geo_data, {'Fault1': 'fault1', 'Fault2': 'fault2', 'Fault3': 'fault3',
                         'Rest': ['layer1', 'layer2', 'layer3']}, verbose=1
             
             )

gp.get_data(geo_data, 'series')

In [ ]:
interp_data = gp.InterpolatorData(geo_data, compile_theano=True,
                                  theano_optimizer='fast_compile',
                                  is_lith=True, is_fault=True)

In [ ]:
import copy
geo_data_i = copy.copy(geo_data)

In [ ]:
# We instanciate a vtkPlot objecth
vtk_plot = gp.plotting.vtkPlot(geo_data_i)

# We call the real time plotter passing thhe interp_data
vtk_plot.plot_surfaces_3D_real_time(interp_data, size=(1920, 2160))

In [ ]:
vtk_plot.observe_df(geo_data_i, itype='surface_points')

In [ ]:
vtk_plot.observe_df(geo_data_i, itype='orientations')

In [ ]:
vtk_plot.observe_df(geo_data_i, itype='faults_relations')

In [ ]:
b

In [ ]:
vtk_plot.resume()

In [ ]:
vtk_plot.close()

#### Saving geo_data

In [ ]:
gp.set_surface_points(geo_data, geo_data_i.surface_points)
gp.set_orientations(geo_data, geo_data_i.orientations)
geo_data.data_to_pickle('backup2.pickle')

In [ ]:
lith_block, fault_block = gp.compute_model(interp_data)

In [ ]:
%matplotlib notebook
gp.plot_scalar_field(geo_data_i, lith_block[1], 25, plot_data=True)

In [ ]:
%matplotlib notebook
gp.plot_section(geo_data_i, fault_block[0], 25, plot_data=True)